# Cjr2185: Burner email service projcet

## People:
### Name: Christophe Rimann
### Uni: cjr2185
### Solo project

## What it does

There are two main components to my project. The first is the actual flask server, located at app.py. The second is a series of scripts that prepare the user's environment for the server's startup. I found this second component to be far more challenging, and am much more proud of it as a result.

### Flask server (app.py)
I was quite frustrated with the state of burner email address services (services where you could create a new email specific for one use, i.e. to recieve a signup email etc): they all either are not very easy to create, or are not private, or automatically delete your address and associated emails after a period of not logging on. As such, I decided to create a tool that fixed these issues. It works as follows. First, the user signs up (currently only an email is required as the only form of authentication, no password), performed in the signupUser function. The user can login as well once the user has been created, which uses the loginUser function to check if the user has registered previously. Both login and signup save a cookie with the current user, in order to authenticate the user later on. The user is then presented with a list of their burners, as well as all the emails they've recieved to these burners (the userHome function). They can click on individual emails to see details about the email (calling the viewemail function) or click on a burner to view all emails received to just that burner (calls viewaddress). From each of these pages, the user can delete an email (calls deleteemail) or a burner (calls deleteaddress) respectively. The user can create a new burner, which calls the newburn function to add it to the database. At the end, the user can logout, which clears the cookie. The flask server interfaces with the mailgun API by exposing a webhook, which when triggered will save the email triggering it to the database, associating them with a given burner (via the processmail function). The database used was sqlite, as this seemed to be the easiest to setup and least invasive on the tester's environment. 

### Startup scripts
In order to be able to expose a webhook, I needed for my application to be on the web, but I still wanted it to be easily testable, so I built two startup scripts: run and setup to help with this. Most of both is the same logic (they utilize the Server class I define in server_helpers.py), but setup also includes some aditional logic. Both run and setup have shebangs so they can be run as startup scripts. Setup creates a database file using sqlite, and then sets up the necessary tables. Then it creates an instance of the server class in server_helpers and runs it. Run just creates the server instance and runs it; it doesn't touch the database. Server does the following: it first registers signal exiters so that if you quit at any point all background processes will be killed as well. Then, it starts an instance of flask in the background, exporting the flask environmental variable as well so that flask knows how to run. It then starts ngrok (a prepackaged script I found - it does so by trying ngrok for mac, then linux, then windows) in a separate background process to create a tunnel between the port flask is running on and the open web, so that mailgun has a place to send its webhook requests. I used ngrok because thats what stackoverflow seemed to suggest was the best solution. Next, the program gets the ngrok url using two methods (I did it two ways because I was curious to see how each way worked) - by parsing the web interface of ngrok using beautiful soup, and by sending a request to the ngrok API using the requests library. It then compares them to make sure they're the same, and then it sends this url to mailgun via the requests library so that mailgun knows where to send new emails. If at any point the user hits ctrl-c, the program captures this input and kills both the flask and ngrok background processes before dying.

### Running
** This program depends on ports 5000 and 4040 being free. Please make sure those ports are free prior to running.** In order to make sure that there is little to no effect on the host operating system, I only used packages that are bundled with the anaconda distribution we were told to use. If for some reason you are not using the anaconda distribution, you can install the required packages by running \$ pip3 install -r requirements.txt, which seem to be the way google suggested handling requirements. Next, ** please make sure that you have execution privileges ** for both setup and run. After that, you should only have to run \$ ./setup, which will also trigger ./run. On subsequent runs, i.e. if you've quit the server but want to restart it, just run $ ./run. DO NOT RERUN ./setup to restart the server. If you want to rerun ./setup and start from scratch, please delete the t1.db file first. 

I have bundled copies of ./ngrok for windows, mac, and linux, but I have only been able to test on mac and linux given that I do not have access to a windows operating system. I've tested everything on macOS Sierra v 10.12.3, Ubuntu 16.04, and Debian 8.7 (the latter two on clean installs on Digital Ocean with free github education credit!), and it worked without issue with \$ pip3 install -r requirements.txt followed by \$ ./setup. I've left an instance running as well and will leave it up for the next month (until * at least * May 6th) so you can easily test functionality there as well; to get the url please go to http://www.columbia.edu/~cjr2185/3101url.html (this way I can update the url should the server die).

## Evaluation

I found python to be an excellent choice for my program. I'd previously used JSP and JavaBeans for web development at my internship, and almost everything about python and flask was easier. Firstly, requests as a library was so much easier for me to use than Java's IO suite. BeautifulSoup was also incredibly easy to use (although I've never actually used JSoup so I'm not an expert). String manipulation, which I found to be very important for database writes/presenting things to the user/dozens of other things in web development was also much easier than having to call an instance of stringbuilder every time. Interacting with the database required much lower overhead than it does in Java. Also, defaultdict was a godsend for database queries - not having to check if the query was empty first and being able to iterate through and add to an empty dictionary made my life much easier. In fact, I had originally only aimed to build the flask server and not the script because my previous experience with JavaBeans thought that that would be a very time intensive task, and it was only when it took less time than expected that I decided I would delve into multiprocessing, something I'd never really worked with before except in the context of this class. I had some difficulty getting the multiprocessing/signaling/environments to work, but I think that was more due to my inexperience rather than anything else. All in all, I found python to just require significantly less overhead in the amount of code I was writing; I was very pleased with how it performed and will almost certainly be building more webapps with it soon.